In [6]:
import yfinance as yf
import numpy as np
import pandas as pd
from typing import Tuple
from scipy.optimize import minimize
from datetime import datetime
import bisect
import warnings
from scipy.integrate import quad, IntegrationWarning
import functools
import time
from numba import njit, prange, float64, complex128

# Suppress specific warnings from scipy.integrate.quad
warnings.filterwarnings('ignore', category=UserWarning, message='The integral is probably divergent, or slowly convergent.')
warnings.filterwarnings('ignore', category=IntegrationWarning)

# Keep get_data_Calibration unchanged

def get_data_Calibration(
    symbol: str,
    target_expiration: str,  # Format 'YYYY-MM-DD'
    underlying_price: float,
) -> pd.DataFrame:
    try:
        # Fixed tuple definitions - remove the trailing commas
        min_volume: int = 5
        max_spread_pct: float = 10.0
        moneyness_range: Tuple[float, float] = (0.85, 1.15)
        num_expirations: int = 8  # Number of expirations we want on each side
        
        ticker = yf.Ticker(symbol)
        all_expirations = ticker.options
        result = []

        # Convert all expirations to datetime for comparison
        expiration_dates = [pd.to_datetime(exp).date() for exp in all_expirations]
        target_date = pd.to_datetime(target_expiration).date()
        
        # Find the index of the closest expiration to the target date
        closest_idx = bisect.bisect_left(expiration_dates, target_date)
        if closest_idx >= len(expiration_dates):
            closest_idx = len(expiration_dates) - 1
            
        # If the closest date is after the target, look one before
        if closest_idx > 0 and (abs((expiration_dates[closest_idx-1] - target_date).days) < 
                               abs((expiration_dates[closest_idx] - target_date).days)):
            closest_idx -= 1
        
        # Calculate how many dates we can get before the target
        available_before = closest_idx
        # Calculate how many dates we can get after the target (including the closest)
        available_after = len(expiration_dates) - closest_idx
        

        # Calculate how many we should get from each side
        to_take_before = min(available_before, num_expirations)
        to_take_after = min(available_after, num_expirations)
        
        # If we can't get enough from one side, get more from the other side
        extra_before = 0
        extra_after = 0
        
        if to_take_before < num_expirations:
            # We need to get extra from after side
            extra_after = min(available_after - to_take_after, num_expirations - to_take_before)
        
        if to_take_after < num_expirations:
            # We need to get extra from before side
            extra_before = min(available_before - to_take_before, num_expirations - to_take_after)
            
        # Calculate final indices
        start_idx = max(0, closest_idx - to_take_before - extra_before)
        end_idx = min(len(expiration_dates), closest_idx + to_take_after + extra_after)
        
        # Get the selected expirations
        selected_expirations = all_expirations[start_idx:end_idx]


        for expiration in selected_expirations:

            options = ticker.option_chain(expiration).calls

            options = options[
                ["strike", "bid", "ask", "volume", "openInterest", "impliedVolatility"]
            ].dropna()

            options["spread"] = options["ask"] - options["bid"]
            options["spreadPct"] = (
                options["spread"] / ((options["bid"] + options["ask"]) / 2)
            ) * 100

            options = options[
                (options["bid"] > 0)
                & (options["ask"] > 0)
                & (options["volume"] >= min_volume)
                & (options["openInterest"] > 0)
                & (options["spreadPct"] <= max_spread_pct)
            ]

            moneyness = options["strike"] / underlying_price
            options = options[
                (moneyness >= moneyness_range[0]) & (moneyness <= moneyness_range[1])
            ]
            
            options = options[(options["impliedVolatility"] > 0.01) & (options["impliedVolatility"] < 3)]

            # Calculate maturity
            today = datetime.today().date()
            expiration_date = pd.to_datetime(expiration).date()
            maturity = (expiration_date - today).days / 365.25

            for _, row in options.iterrows():
                mid = (row["bid"] + row["ask"]) / 2
                result.append(
                    {
                        "mid_price": mid,
                        "ask_price": row["ask"],
                        "maturity": maturity,
                        "strike": row["strike"],
                        "implied_volatility": row["impliedVolatility"],
                        "volume": row["volume"],
                        "expiration": expiration_date,
                        "moneyness": row["strike"] / underlying_price
                    }
                )

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(result)
        
        # Sort by expiration and strike
        if not df.empty:
            df = df.sort_values(['expiration', 'strike'])
            
        return df

    except Exception as e:
        raise RuntimeError(f"Failed to fetch market data: {str(e)}")
    
# test the function
if __name__ == "__main__":
    symbol = "^SPX"
    target_expiration = "2025-07-31"
    underlying_price = 5525.21
    data = get_data_Calibration(symbol, target_expiration, underlying_price)
    print(f"Data shape: {data.shape}")
    print(f"Columns in data: {data.columns.tolist()}")

Data shape: (1020, 8)
Columns in data: ['mid_price', 'ask_price', 'maturity', 'strike', 'implied_volatility', 'volume', 'expiration', 'moneyness']


In [7]:
# Global price cache to reuse calculations across iterations
PRICE_CACHE = {}

@njit(complex128(float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64))
def heston_cf_core(phi, x, r, T, kappa, rho, volvol, theta, var0, div, b, u):
    """Core calculations for Heston characteristic function - matches original implementation exactly"""
    a = kappa * theta
    rvpj = rho * volvol * phi * complex(0, 1)
    
    # Use more stable computations for d
    daux = (b - rvpj) ** 2 - volvol**2 * (2 * u * phi * complex(0, 1) - phi**2)
    d = np.sqrt(daux)
    
    # Handle g calculation with numerical stability
    num = b - rvpj + d
    den = b - rvpj - d
    
    # Avoid division by zero for g
    if abs(den) < 1e-15:
        g = 0.0
    else:
        g = num / den
    
    # Handle exponential carefully to avoid overflow
    if np.real(d * T) > 700:
        exp_dT = np.inf
    else:
        exp_dT = np.exp(d * T)
    
    g_exp_dT = g * exp_dT
    
    # Safe computation for D
    if abs(1.0 - g_exp_dT) < 1e-15:
        D = (num / volvol**2) * T
    else:
        if np.real(d * T) > 700:
            exp_term = np.inf
        else:
            exp_term = np.exp(d * T)
        
        D = (num / volvol**2) * ((1.0 - exp_term) / (1.0 - g * exp_term))
    
    # Safe computation for C
    if abs(1.0 - g) < 1e-15:
        log_term = d * T
    else:
        if abs(1.0 - g_exp_dT) < 1e-15:
            # When denominator is near zero
            if np.real(d * T) > 0:
                log_term = 700.0  # Large positive number instead of inf
            else:
                log_term = -700.0  # Large negative number instead of -inf
        else:
            # More stable logarithm calculation
            log_term = np.log((1.0 - g_exp_dT) / (1.0 - g))
    
    # Final calculation of C
    C = (r - div) * phi * T * complex(0, 1) + (a / volvol**2) * ((b - rvpj + d) * T - 2.0 * log_term)
    
    # Final CF
    CF = np.exp(x * phi * complex(0, 1) + C + D * var0)
    
    return CF

def integrand_core(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    """Core calculations for the integrand function"""
    if abs(phi) < 1e-15:
        return 0.0
    
    x = np.log(S)
    u = 0.5 if P1P2 == 1 else -0.5
    b = kappa - rho * volvol if P1P2 == 1 else kappa
    
    try:
        CF = heston_cf_core(phi, x, r, T, kappa, rho, volvol, theta, var0, div, b, u)
        if np.isnan(CF) or np.isinf(CF):
            return 0.0
        
        Output = np.real((np.exp(-phi * np.log(K) * complex(0, 1)) * CF) / (phi * complex(0, 1)))
        if np.isnan(Output) or np.isinf(Output):
            return 0.0
            
        return Output
    except:
        return 0.0

def NumIntegration(aLim, bLim, nDiv, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    """Numerical integration using Simpson's rule - matches original implementation"""
    if nDiv % 2 != 0:  # ensure nDiv is even
        nDiv += 1
    
    Delta = (bLim - aLim) / nDiv
    EveryX = np.linspace(aLim, bLim, nDiv + 1)
    EveryY = np.zeros(nDiv + 1)
    
    # Calculate function values at each point
    for i in range(nDiv + 1):
        EveryY[i] = integrand_core(EveryX[i], S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2)
    
    # Simpson's rule integration
    even_indices = np.arange(2, nDiv, 2)
    odd_indices = np.arange(1, nDiv, 2)
    
    even_sum = np.sum(EveryY[even_indices])
    odd_sum = np.sum(EveryY[odd_indices])
    
    Integral = (Delta/3) * (EveryY[0] + 4*odd_sum + 2*even_sum + EveryY[-1])
    
    return Integral

@functools.lru_cache(maxsize=20000)
def P1P2Heston(S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    """P1 and P2 calculation with adaptive integration - matches original implementation"""
    cache_key = (S, K, T, r, round(kappa, 6), round(rho, 6), 
                round(volvol, 6), round(theta, 6), round(var0, 6), 
                round(div, 6), P1P2)
    
    if cache_key in PRICE_CACHE:
        return PRICE_CACHE[cache_key]
    
    # Use an appropriate number of points based on maturity
    if T < 0.1:  # For very short maturities, we need more points
        nDiv = 2000
    elif T < 0.5:  # For short maturities
        nDiv = 1000
    else:  # For longer maturities
        nDiv = 500
    
    # For extreme strikes, use more points
    moneyness = S/K
    if moneyness < 0.8 or moneyness > 1.2:
        nDiv *= 2
    
    # Use appropriate upper limit
    upper_limit = min(100, max(50, 100/T))
    
    # Perform numerical integration
    NumInt = NumIntegration(1e-6, upper_limit, nDiv, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2)
    
    # Check if the result seems reasonable
    PP = 0.5 + (1 / np.pi) * NumInt
    
    # Fall back to quad if result is outside valid range
    if PP < 0 or PP > 1 or np.isnan(PP):
        try:
            integral, _ = quad(integrand_core, 1e-6, upper_limit,
                              args=(S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2),
                              limit=100, epsabs=1e-8, epsrel=1e-8)
            PP = 0.5 + (1 / np.pi) * integral
        except:
            PP = 0.5  # Fallback value
    
    # Make sure result is between 0 and 1
    PP = max(0.0, min(1.0, PP))
    PRICE_CACHE[cache_key] = PP
    return PP

@functools.lru_cache(maxsize=10000)
def CallHestonCForm_Cached(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Heston call option pricing with dividend - matches original implementation"""
    # Ensure parameters are within valid ranges
    kappa = max(0.001, kappa)  # Mean reversion speed must be positive
    volvol = max(0.001, volvol)  # Vol of vol must be positive
    theta = max(0.0001, theta)  # Long-term variance must be positive
    var0 = max(0.0001, var0)    # Initial variance must be positive
    rho = max(-0.999, min(0.999, rho))  # Correlation must be between -1 and 1
    
    # Fast path for very short maturities
    if T <= 1e-6:
        return max(0, S * np.exp(-div * T) - K * np.exp(-r * T))
    
    P1 = P1P2Heston(S, K, T, r, kappa, rho, volvol, theta, var0, div, 1)
    P2 = P1P2Heston(S, K, T, r, kappa, rho, volvol, theta, var0, div, 2)
    
    # Option price calculation with checks
    CallValue = S * np.exp(-div * T) * P1 - K * np.exp(-r * T) * P2
    
    # Ensure non-negative option price
    return max(0.0, CallValue)

def CallHestonCForm(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Heston call option pricing wrapper with global caching"""
    # Check if calculation is already in the global cache
    cache_key = (S, K, T, r, round(kappa, 6), round(rho, 6), 
                round(volvol, 6), round(theta, 6), round(var0, 6), round(div, 6))
    
    if cache_key in PRICE_CACHE:
        return PRICE_CACHE[cache_key]
    
    result = CallHestonCForm_Cached(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    PRICE_CACHE[cache_key] = result
    return result

def PutHestonCForm(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Put option pricing function using put-call parity"""
    CallValue = CallHestonCForm(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    PutValue = CallValue - S * np.exp(-div * T) + K * np.exp(-r * T)
    
    # Ensure non-negative option price
    return max(0.0, PutValue)

def clear_cache():
    """Clear all caches to free memory"""
    global PRICE_CACHE
    PRICE_CACHE = {}
    P1P2Heston.cache_clear()
    CallHestonCForm_Cached.cache_clear()


# Example usage
if __name__ == "__main__":
    # Parameters
    S = 100         # Initial stock price
    K = 101         # Strike price
    T = 1           # Time to maturity (years)
    r = 0.02        # Risk-free rate
    kappa = 1.5     # Mean reversion speed
    rho = -0.4      # Correlation
    volvol = 0.6    # Volatility of variance
    theta = 0.03    # Long-term variance mean
    var0 = 0.014    # Initial variance
    div = 0.005     # Dividend yield
    
    # Calculate prices using semi-analytical method
    callPrice = CallHestonCForm(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    
    print("Semi-Analytical Heston Model Results:")
    print(f"Call Option Price: {callPrice:.4f} USD")

Semi-Analytical Heston Model Results:
Call Option Price: 5.1571 USD


In [8]:
# Parallel version for batch pricing
def heston_prices_parallel(params, Spots, Strikes, Maturities, Rates, div):
    """Price multiple options in parallel for calibration"""

    results = []
    kappa, rho, volvol, theta, var0 = params  # Unpack parameters
    
    for i in range(len(Spots)):
        # Call with proper arguments in correct order
        price = CallHestonCForm(Spots[i], Strikes[i], Maturities[i], Rates[i], 
                               kappa, rho, volvol, theta, var0, div)
        results.append(price)
        
    return np.array(results)


def OptFunctionFast(params, Spots, Maturities, Rates, Strikes, MarketP, div, check_bounds=True):
    """Optimized cost function for faster calibration with early stopping"""
    kappa, rho, volvol, theta, var0 = params
    min_price = 1e-6
    error_penalty = 1e10
    
    # Fast boundary check - return early for invalid parameters
    if check_bounds and not (
            0.1 <= kappa <= 15.0 and 
            -0.99 <= rho <= 0.0 and 
            0.01 <= volvol <= 2.0 and 
            0.001 <= theta <= 0.5 and 
            0.001 <= var0 <= 0.5):
        return error_penalty
    
    # Filter valid market prices
    valid_indices = np.isfinite(MarketP) & (MarketP > 0)
    if not np.any(valid_indices):
        return error_penalty
        
    # Use only valid data points
    valid_Spots = Spots[valid_indices]
    valid_Strikes = Strikes[valid_indices]
    valid_Maturities = Maturities[valid_indices]
    valid_Rates = Rates[valid_indices]
    valid_MarketP = MarketP[valid_indices]
    
    # Calculate model prices without threading for small datasets
    model_prices = heston_prices_parallel(
        params, valid_Spots, valid_Strikes, valid_Maturities, valid_Rates, div
    )
    
    # Calculate simple error metric - optimization for speed
    abs_errors = np.abs(model_prices - valid_MarketP)
    mean_error = np.mean(abs_errors)
    
    # Check threshold for early stopping - if error is already large, return early
    if mean_error > 10.0:  # If average error is $10 or more, skip detailed calculations
        return mean_error * 10  # Simple penalty
    
    # ATM options have higher weights
    moneyness = valid_Strikes / valid_Spots
    weights = 1.0 + np.exp(-20.0 * (moneyness - 1.0) ** 2)  # Simplified weighting
    
    # Weighted squared error
    weighted_errors = (model_prices - valid_MarketP) ** 2 * weights
    mse = np.mean(weighted_errors)
    
    return mse if np.isfinite(mse) else error_penalty

In [9]:
def fast_calibrate_heston(
    symbol: str,
    expiration: str,
    underlying_price: float,
    risk_free_rate: float,
    dividend_yield: float = 0.01,  # Added explicit dividend_yield parameter
    max_time_seconds: int = 60,  # Reduced from 120 to 60 seconds
):
    """Fast Heston calibration for small datasets with aggressive optimizations"""
    try:
        start_time = time.time()
        clear_cache()  # Clear cache before starting new calibration
        
        # Get market data
        market_data = get_data_Calibration(symbol, expiration, underlying_price)
        
        if market_data.empty:
            raise ValueError("No valid market data available for calibration")

        # Use mid_price for calibration target
        MarketP = market_data["mid_price"].values 
        Strikes = market_data["strike"].values
        Maturities = market_data["maturity"].values
        Rates = np.full_like(Maturities, risk_free_rate)
        Spots = np.full_like(Maturities, underlying_price)

        # Filter options data - focus only on valid, liquid options
        valid_indices = (MarketP > 0) & (Maturities > 0) & np.isfinite(MarketP)
        if not np.any(valid_indices):
             raise ValueError("No market data points with positive price and maturity.")
             
        MarketP = MarketP[valid_indices]
        Strikes = Strikes[valid_indices]
        Maturities = Maturities[valid_indices]
        Rates = Rates[valid_indices]
        Spots = Spots[valid_indices]
        filtered_market_data = market_data[valid_indices].copy()  # Use copy to avoid SettingWithCopyWarning

        # Subsample data if we have too many options for faster calibration
        n_options = len(MarketP)
        if n_options > 100:  # If more than 50 options, take a representative subset
            try:
                # Group by moneyness and maturity to get a representative subset
                filtered_market_data['moneyness_bin'] = pd.qcut(filtered_market_data['moneyness'], 5, labels=False, duplicates='drop')
                filtered_market_data['maturity_bin'] = pd.qcut(filtered_market_data['maturity'], 
                                                           min(5, len(filtered_market_data['maturity'].unique())), 
                                                           labels=False, duplicates='drop')
                
                # Create a stratified sample
                grouped = filtered_market_data.groupby(['moneyness_bin', 'maturity_bin'])
                sampled_data = pd.DataFrame()
                
                # Take a few options from each group
                for _, group in grouped:
                    sample_size = min(3, len(group))  # Take up to 3 from each group
                    sampled_data = pd.concat([sampled_data, group.sample(sample_size)])
                
                if len(sampled_data) > 0:
                    filtered_market_data = sampled_data
                    MarketP = filtered_market_data["mid_price"].values
                    Strikes = filtered_market_data["strike"].values
                    Maturities = filtered_market_data["maturity"].values
                    Rates = np.full_like(Maturities, risk_free_rate)
                    Spots = np.full_like(Maturities, underlying_price)
                    print(f"Reduced from {n_options} to {len(MarketP)} options for faster calibration")
            except Exception as e:
                # Fallback to simple random sampling if quantile binning fails
                print(f"Stratified sampling failed: {e}. Using random sampling instead.")
                sample_size = min(50, n_options)
                sampled_indices = np.random.choice(n_options, size=sample_size, replace=False)
                filtered_market_data = filtered_market_data.iloc[sampled_indices]
                MarketP = filtered_market_data["mid_price"].values
                Strikes = filtered_market_data["strike"].values
                Maturities = filtered_market_data["maturity"].values
                Rates = np.full_like(Maturities, risk_free_rate)
                Spots = np.full_like(Maturities, underlying_price)
                print(f"Reduced from {n_options} to {len(MarketP)} options using random sampling")

        # Smart initial estimate based on market data
        avg_iv = np.mean(filtered_market_data["implied_volatility"])
        var0 = avg_iv ** 2
        theta = var0
        kappa = 1.5
        volvol = 0.3 * avg_iv  # Proportional to avg IV
        rho = -0.7
        
        # Multiple initial guesses for better convergence
        initial_guesses = [
            [kappa, rho, volvol, theta, var0],  # Base guess
            [3.0, -0.5, 0.5, theta, var0],      # Alternative 1
            [1.0, -0.8, 0.2, theta, var0]       # Alternative 2
        ]
        
        # Parameter bounds - tighter for faster convergence
        bounds = [
            (0.1, 10.0),     # kappa: reduced upper bound
            (-0.95, 0.0),    # rho: typically negative for equity
            (0.01, 1.5),     # volvol: reduced upper bound
            (0.001, 0.4),    # theta: tighter range
            (0.001, 0.4)     # var0: tighter range
        ]
        
        # Optimization setup with dividend yield parameter
        opt_args = (Spots, Maturities, Rates, Strikes, MarketP, dividend_yield, True)
        best_result = None
        best_error = float('inf')
        
        # Try different initial guesses with a fast local search
        for x0 in initial_guesses:
            # Check time budget
            if time.time() - start_time > max_time_seconds:
                print(f"Time limit reached after trying {initial_guesses.index(x0)} initial points")
                break
                
            # Fast optimization with limited iterations
            try:
                result = minimize(
                    OptFunctionFast,
                    x0,
                    method="L-BFGS-B",
                    bounds=bounds,
                    args=opt_args,
                    options={
                        "maxiter": 50,     # Reduced from 200 to 50
                        "maxfun": 100,    # Limit function evaluations
                        "disp": False,
                        "ftol": 1e-6,     # Reduced precision
                        "gtol": 1e-5      # Reduced precision
                    }
                )
                
                # Check if this result is better than previous ones
                if result.fun < best_error:
                    best_result = result
                    best_error = result.fun
            except Exception as e:
                print(f"Optimization failed for initial guess {x0}: {e}")
                continue
        
        # If we have no valid result, try one more time with Nelder-Mead (more robust)
        if best_result is None:
            try:
                # Nelder-Mead doesn't use bounds but is more robust
                result = minimize(
                    lambda p: OptFunctionFast(p, Spots, Maturities, Rates, Strikes, MarketP, dividend_yield, False),
                    initial_guesses[0],
                    method="Nelder-Mead",
                    options={
                        "maxiter": 100,
                        "maxfev": 200,
                        "disp": False,
                        "adaptive": True
                    }
                )
                best_result = result
            except Exception as e:
                raise ValueError(f"All optimization attempts failed: {e}")
        
        # Extract calibrated parameters
        calibrated_params = {
            "kappa": float(best_result.x[0]),
            "rho": float(best_result.x[1]),
            "volvol": float(best_result.x[2]),
            "theta": float(best_result.x[3]),
            "var0": float(best_result.x[4])
        }
        
        # Evaluate model prices with calibrated parameters
        model_prices = heston_prices_parallel(
            (calibrated_params["kappa"], calibrated_params["rho"], 
             calibrated_params["volvol"], calibrated_params["theta"], calibrated_params["var0"]),
            Spots, Strikes, Maturities, Rates, dividend_yield  # Added dividend_yield
        )
        
        # Calculate error metrics using the full dataset (not just the sample)
        valid_model_prices = np.isfinite(model_prices) & (model_prices > 0)
        MarketP_valid = MarketP[valid_model_prices]
        model_prices_valid = model_prices[valid_model_prices]
        filtered_market_data_valid = filtered_market_data.iloc[valid_model_prices]
        
        if len(MarketP_valid) == 0:
            raise ValueError("No valid model prices could be calculated with calibrated parameters.")

        # Calculate basic error metrics
        errors = model_prices_valid - MarketP_valid
        relative_errors_pct = (errors / MarketP_valid) * 100
        mse = np.mean(errors**2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(errors))
        total_time = time.time() - start_time

        return {
            "success": best_result.success if hasattr(best_result, 'success') else True,
            "kappa": float(calibrated_params["kappa"]),
            "theta": float(calibrated_params["theta"]),
            "volvol": float(calibrated_params["volvol"]),
            "rho": float(calibrated_params["rho"]),
            "var0": float(calibrated_params["var0"]),
            "div": float(dividend_yield),  # Include div in result
            "calibration_metrics": {
                "MSE": mse,
                "RMSE": rmse,
                "MAE": mae,
                "max_abs_error": np.max(np.abs(errors)),
                "mean_rel_error_pct": np.mean(np.abs(relative_errors_pct)),
                "median_rel_error_pct": np.median(np.abs(relative_errors_pct)),
                "n_options_used": len(MarketP_valid),
                "original_n_options": n_options,
                "optimizer_iterations": best_result.nit if hasattr(best_result, 'nit') else best_result.nfev,
                "calibration_time_seconds": total_time
            },
            "market_data_used": filtered_market_data_valid.to_dict(orient="records")
        }

    except Exception as e:
        import traceback
        return {
            "success": False,
            "error": str(e),
            "error_details": traceback.format_exc(),
            "kappa": 0.0,
            "theta": 0.0,
            "volvol": 0.0,
            "rho": 0.0,
            "var0": 0.0,
            "div": float(dividend_yield),
            "calibration_time_seconds": time.time() - start_time
        }

In [10]:
# Test the ultra-fast calibration function
symbol = '^SPX'  # Example symbol
target_expiration = '2025-07-31'  # Example target expiration
underlying_price = 5525.21  # Example underlying price
risk_free_rate = 0.04  # Example risk-free rate (e.g., 4%)
dividend_yield = 0.01  # Example dividend yield (e.g., 1%)

print(f"Starting fast calibration for {symbol}...")
print(f"Target: {target_expiration}, S0: ${underlying_price:.2f}, r: {risk_free_rate:.2%}, div: {dividend_yield:.2%}")

# Use a timer to measure calibration speed
t0 = time.time()

calibration_results = fast_calibrate_heston(
    symbol,
    target_expiration,
    underlying_price,
    risk_free_rate,
    dividend_yield,  # Added dividend yield parameter
    max_time_seconds=60  # Force completion within 60 seconds
)

calibration_time = time.time() - t0
print(f"\nCalibration completed in {calibration_time:.2f} seconds")

if calibration_results['success']:
    print("\nCalibration Results:")
    print("===================")
    
    print("\nCalibrated Heston Parameters:")
    print(f"  κ (kappa)  = {calibration_results['kappa']:.4f}  - Mean reversion rate")
    print(f"  θ (theta)  = {calibration_results['theta']:.4f}  - Long-term variance")
    print(f"  σ (volvol) = {calibration_results['volvol']:.4f}  - Volatility of variance")
    print(f"  ρ (rho)    = {calibration_results['rho']:.4f}  - Correlation")
    print(f"  v₀ (var0)  = {calibration_results['var0']:.4f}  - Initial variance")
    print(f"  δ (div)    = {calibration_results['div']:.4f}  - Dividend yield")
    
    # Calculate volatilities
    print(f"  Long-term volatility = {np.sqrt(calibration_results['theta']):.2%}")
    print(f"  Initial volatility   = {np.sqrt(calibration_results['var0']):.2%}")
    
    print("\nCalibration Metrics:")
    metrics = calibration_results['calibration_metrics']
    if 'original_n_options' in metrics and metrics['original_n_options'] != metrics['n_options_used']:
        print(f"  Data sample: {metrics['n_options_used']} of {metrics['original_n_options']} options")
    else:
        print(f"  Options used: {metrics.get('n_options_used', 'N/A')}")
        
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAE: {metrics['MAE']:.4f}")
    print(f"  Max Error: ${metrics.get('max_abs_error', 'N/A'):.2f}")
    print(f"  Mean Relative Error: {metrics.get('mean_rel_error_pct', 'N/A'):.2f}%")
    
    # Check Feller condition
    feller = 2 * calibration_results['kappa'] * calibration_results['theta'] - calibration_results['volvol']**2
    print(f"\nFeller Condition: {feller:.4f} {'(satisfied)' if feller > 0 else '(violated)'}")
else:
    print("\nCalibration Failed.")
    print(f"Error: {calibration_results.get('error', 'Unknown error')}")
    if 'error_details' in calibration_results:
        print("\nError Details:")
        print(calibration_results['error_details'])

Starting fast calibration for ^SPX...
Target: 2025-07-31, S0: $5525.21, r: 4.00%, div: 1.00%
Reduced from 1020 to 75 options for faster calibration

Calibration completed in 9.77 seconds

Calibration Results:

Calibrated Heston Parameters:
  κ (kappa)  = 3.0000  - Mean reversion rate
  θ (theta)  = 0.0546  - Long-term variance
  σ (volvol) = 0.5000  - Volatility of variance
  ρ (rho)    = -0.5000  - Correlation
  v₀ (var0)  = 0.0546  - Initial variance
  δ (div)    = 0.0100  - Dividend yield
  Long-term volatility = 23.37%
  Initial volatility   = 23.37%

Calibration Metrics:
  Data sample: 75 of 1020 options
  RMSE: 65.2496
  MAE: 28.5796
  Max Error: $517.48
  Mean Relative Error: 39.69%

Feller Condition: 0.0777 (satisfied)


## Performance Optimization Notes

The fast calibration implementation includes several key optimizations:

1. **Reduced Integration Points**: Using fewer Gauss-Laguerre quadrature points (16 instead of 48)

2. **Multi-level Caching**: 
   - LRU cache for characteristic function
   - LRU cache for option pricing
   - Global dictionary cache to avoid repeated calculations

3. **Fast Approximations**:
   - Black-Scholes approximation for low volatility of volatility
   - Early exit for deep OTM or extreme-strike options

4. **Optimization Strategy**:
   - Multiple starting points instead of global optimization
   - Limited iterations and function evaluations
   - Early stopping when error is already large

5. **Data Sampling**:
   - For larger datasets, uses a stratified sample of options
   - Ensures representation across moneyness and maturities
   
6. **Simplified Error Calculation**:
   - Fast weighted error calculation
   - Early exit for obviously poor parameter sets

These optimizations significantly reduce the calibration time while maintaining reasonable accuracy.